
 Preprocessing the dataset to feed the text similarity conveyor

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
import math
import random
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

messages = pd.read_csv('/content/drive/MyDrive/movie-corpus/movie_lines_normalized.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

Mounted at /content/drive


In [ ]:
messages.columns = ['msg_line', 'user_id', 'movie_id', 'msg']

In [ ]:
messages.head(10)

,msg_line,user_id,movie_id,msg
0,L1045,u0,m0,They do not!
1,L1044,u2,m0,They do to!
2,L985,u0,m0,I hope so.
3,L984,u2,m0,She okay?
4,L925,u0,m0,Let's go.
5,L924,u2,m0,Wow
6,L872,u0,m0,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,No
8,L870,u0,m0,"""""""I'm kidding. You know how sometimes you jus..."
9,L869,u0,m0,Like my fear of wearing pastels?


Cleaning the msg_line of the conversations

In [ ]:
#remove characters
def remove_char(txt):
    return re.sub('[^0-9]','', txt)

In [ ]:

#leaving just the number of the index, so L872 changes to 872
messages['msg_line_clean'] = [remove_char(msg) for msg in messages['msg_line']]

In [ ]:
#change the column type to number
messages['msg_line_clean'] = pd.to_numeric(messages['msg_line_clean'])

In [ ]:
messages = messages.sort_values(by=['msg_line_clean'])

In [ ]:
#set the column as the index
messages = messages.set_index('msg_line_clean')

In [ ]:
messages.head(10)

,msg_line,user_id,movie_id,msg
msg_line_clean,,,,
49,L49,u0,m0,Did you change your hair?
50,L50,u3,m0,No.
51,L51,u0,m0,You might wanna think about it
59,L59,u9,m0,I missed you.
60,L60,u8,m0,It says here you exposed yourself to a group o...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.
62,L62,u8,m0,With the teeth of your zipper?
63,L63,u7,m0,You the new guy?
64,L64,u2,m0,So they tell me...


Removing entities of messages like PEOPLE or ORG

In [ ]:
entities = pd.read_csv('/content/drive/MyDrive/movie-corpus/entity_list_complete.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [ ]:
entities.columns = ['ent', 'type']

In [ ]:

entities['ent'] = entities['ent'].str.lower()

In [ ]:
entities.head()

,ent,type
0,kinda,ORG
1,the dallas times herald,ORG
2,queen louisa,PERSON
3,a.m,GPE
4,cousin hop,PERSON


In [ ]:

entities['ent_len'] = [len(e) for e in entities['ent']]

In [ ]:
s = entities['ent_len'].sort_values(ascending=False).index

In [ ]:

entities = entities.reindex(s)

In [ ]:

entities = entities.reset_index(drop=True)

In [ ]:

entities.head()

,ent,type,ent_len
0,"""""""how can the bolshevik cause gain respect am...",WORK_OF_ART,237
1,"""""""the premier wishes to inform the government...",WORK_OF_ART,192
2,""""""" come tuesday twelve a.m. bingo these like-...",WORK_OF_ART,182
3,"""""""the suggestion of the president regarding t...",WORK_OF_ART,155
4,"""""""the management of boyd's takes pleasure in ...",WORK_OF_ART,146


In [ ]:
data = messages['msg']

In [ ]:

ent_list =  ['PERSON', 'ORG', 'NORP', 'FAC', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']
#ent_list =  ['LANGUAGE']

ent = list()
for i in range(len(entities.index)):
    if entities['type'][i] in ent_list:
        ent.append(entities['ent'][i])

In [ ]:
ent = list(set(ent))
print(len(ent))
print(ent)

27848
['zimnermann', 'windthorne', 'jack w. donald m.', 'sabena', 'joshua strader', 'fate', 'the international house of pancakes', 'level 842con g division of plastic vendable', 'john jay high school', '&lt;&lt;it', 'housekeeping</u', "old seti's", 'missus', 'mongo', '"""making fuck?"', 'crocodile dundee</u', 'katharine clifton', 'b.fl.d.', 'the auditing department of the territorial administrator', "jen'll", 'saito', 'beeeee', 'both', 'frances farmer', 'goddamnit valentine', 'valsney', 'angelica', 'remington party', '""" how nice."', 'ramanujan', '""" heaven on earth?"', 'logjammin', 'flight 180', 'bertrand', 'a tierra del fuegian', 'halt hendricks', 'rather', 'banker', 'fluties pier 17', 'hartley', 'dakota', 'dawn', 'paree', "ss'tangya t'ssorentsa'", '"""charming"', 'providence', 'prefect', 'the war office', '""" oh jack."', 'dennis james rooney', 'the water of life', 'socorro', 'the genetic sector', 'melina hauser', 'the egg bank', 'blue ruin', 'keaton', 'beverly center', 'bob shine

In [ ]:
def remove_entity(corpus):
    corpus = corpus.split(' ')
    corpus = [c for c in corpus if c not in list(dict.keys())]
    return ' '.join(corpus)

In [ ]:
def remove_entity(corpus, entity_list):
    """Removes entities from a corpus.

    Args:
        corpus: The text corpus to process.
        entity_list: A list of entities to remove.

    Returns:
        The corpus with entities removed.
    """
    corpus = corpus.split(' ')
    # Directly checking if the word is in the entity list.
    corpus = [c for c in corpus if c not in entity_list]
    return ' '.join(corpus)

In [ ]:
#Opening conversation sequence dataset

In [ ]:
#read the file with the conversation sequence
conv_seq = pd.read_csv('/content/drive/MyDrive/movie-corpus/movie_conversations.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [ ]:
conv_seq.columns = ['user1_id', 'user2_id', 'movie_id', 'sequence']

In [ ]:
conv_seq.head(10)

,user1_id,user2_id,movie_id,sequence
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


Build conversation sequence

In [ ]:
def split_conversation(txt):
    txt_alt = txt.split(' ')
    return txt_alt

In [ ]:
def seq_to_list(seq):
    seq_list = [remove_char(s) for s in seq]
    return seq_list

In [ ]:

#initializing the msg_2 column
messages['msg_2'] = '-'

In [ ]:
def link_conversations(seq_list, df, filter1, filter2):
    i = 0
    while i in range(len(seq_list)):
        if i+1 < len(seq_list):
            next_msg = df.loc[int(seq_list[i+1]), filter1]
            df.at[int(seq_list[i]), filter2] = next_msg
        i+=1

In [ ]:
#link each message with its answer
for conv in conv_seq['sequence']:
    #split each sequence by space
    seq = split_conversation(conv)

    #remove the char L from the sequences
    txt_alt = [remove_char(s) for s in seq]

    #use the conversation sequence to build the target answer for each message
    link_conversations(txt_alt, messages, 'msg', 'msg_2')

In [ ]:

messages.head(30)

,msg_line,user_id,movie_id,msg,msg_2
msg_line_clean,,,,,
49,L49,u0,m0,Did you change your hair?,No.
50,L50,u3,m0,No.,You might wanna think about it
51,L51,u0,m0,You might wanna think about it,-
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?
62,L62,u8,m0,With the teeth of your zipper?,-
63,L63,u7,m0,You the new guy?,So they tell me...
64,L64,u2,m0,So they tell me...,C'mon. I'm supposed to give you the tour.


In [ ]:

# Pre processing the msg
# Removing messages without answer

messages = messages.drop(messages[messages['msg_2'] == '-'].index)
messages = messages.drop(messages[messages['msg_2'] == ''].index)
messages = messages.drop(messages[messages['msg_2'] == 'UNKNOWN'].index)

In [ ]:
data = messages['msg']

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')

import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
def pre_processing_text(corpus):
    #remove html tags
    corpus = re.sub(r'<.*?>', '', str(corpus))

    #remove non-alphanumeric characters
    corpus = re.sub(r'[^a-z A-Z 0-9 \s]', '', str(corpus))

    #remove duplicated spaces
    corpus = re.sub(r' +', ' ', str(corpus))

    #capitalization
    corpus = corpus.lower()

    #tokenization
    corpus = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)

    #lammatization
    corpus = [lemmatizer.lemmatize(c, get_wordnet_pos(c)) for c in corpus]

    #remove punctuation
    corpus = [t for t in corpus if t not in string.punctuation]

    #remove stopwords
    stopwords_ = stopwords.words("english")
    corpus = [t for t in corpus if t not in stopwords_]

    return ' '.join(corpus)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%%time
data_pre_processed = [pre_processing_text(str(m)) for m in data]
data_pre_processed

CPU times: user 5min 42s, sys: 25.3 s, total: 6min 7s
Wall time: 6min 14s


['change hair',
 '',
 'miss',
 'say expose group freshman girl',
 'bratwurst eat lunch',
 'new guy',
 'tell',
 'dakota',
 'north actually howd',
 'kid people actually live',
 'yeah couple outnumber cow though',
 'many people old school',
 'thirtytwo',
 'get',
 'many people go',
 'im use',
 'girl',
 'burn pine perish',
 '',
 'bianca stratford sophomore dont even think',
 '',
 'katarina stratford youve terrorize blaise',
 'express opinion terrorist action',
 'well yes compare choice expression year today event quite mild way bobby rictors gonad retrieval operation go quite well case youre interested',
 'still maintain kick ball merely spectator',
 'tempestuous',
 '',
 'patrick verona random skid',
 'thats pat verona one go year heard porn movie',
 'im sure completely incapable anything interest',
 'always look',
 'mandella eat starve slow way die',
 'whats',
 'realize men fine institution severely lack kill william shakespeare beyond scope normal teenage obsession youre venture far past 

In [ ]:
messages['msg_pre_processed'] = data_pre_processed


Checking and removing duplicated messages in msg field

In [ ]:
data = messages['msg_pre_processed']

In [ ]:
dict = {}
for n in data:
    if n in dict:
        dict[n] = dict[n] + 1
    else:
        dict[n] = 1

In [ ]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
dict

{'': 10465,
 'yes': 1638,
 'yeah': 1169,
 'know': 658,
 'go': 581,
 'dont know': 442,
 'say': 439,
 'whats': 422,
 'mean': 396,
 'right': 393,
 'okay': 386,
 'get': 357,
 'oh': 352,
 'want': 339,
 'well': 338,
 'sure': 330,
 'think': 319,
 'talk': 291,
 'come': 286,
 'hello': 279,
 'really': 267,
 'happen': 256,
 'like': 252,
 'dont': 237,
 'thank': 215,
 'huh': 210,
 'see': 208,
 'tell': 203,
 'hi': 201,
 'im sorry': 198,
 'sir': 192,
 'hell': 183,
 'cant': 181,
 'fuck': 177,
 'yes sir': 174,
 'thanks': 168,
 'nothing': 164,
 'course': 163,
 'unknown': 163,
 'whats wrong': 157,
 'hey': 155,
 'uhhuh': 150,
 'good': 148,
 'thats right': 147,
 'look': 139,
 'im': 136,
 'one': 133,
 'love': 126,
 'excuse': 118,
 'thats': 115,
 'didnt': 111,
 'time': 105,
 'shit': 104,
 'oh god': 102,
 'sorry': 101,
 'dont think': 101,
 'work': 101,
 'way': 99,
 'whats matter': 99,
 'stop': 98,
 'told': 96,
 'guess': 93,
 'dont understand': 93,
 'long': 92,
 'oh yeah': 92,
 'help': 92,
 'fine': 89,
 'much'

In [ ]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'yes']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,
817,L817,u9,m0,Yes,I don't like to do what people expect. Then th...,yes
2571,L2571,u30,m2,Yes.,Please join us. Come on forward.,yes
3968,L3968,u50,m3,Yes?,Well I got the papers on my official up-gradin...,yes
3975,L3975,u50,m3,Yes.,Yesterday I finally called the Accounting Offi...,yes
4334,L4334,u50,m3,Yes it is.,I suppose you're pretty broken up about it?,yes
...,...,...,...,...,...,...
665827,L665827,u9017,m615,Yes.,You're just supposed to repeat what you said.,yes
666075,L666075,u9020,m615,Yes I will.,...But an intellectual relationship like this ...,yes
666154,L666154,u9020,m615,Yes.,What d'ya want to do to kill time?,yes


In [ ]:
#get the repeated messages
d_list = list()
for k in dict:
    if dict[k] > 1:
        d_list.append(k)

In [ ]:
d_list

['',
 'yes',
 'yeah',
 'know',
 'go',
 'dont know',
 'say',
 'whats',
 'mean',
 'right',
 'okay',
 'get',
 'oh',
 'want',
 'well',
 'sure',
 'think',
 'talk',
 'come',
 'hello',
 'really',
 'happen',
 'like',
 'dont',
 'thank',
 'huh',
 'see',
 'tell',
 'hi',
 'im sorry',
 'sir',
 'hell',
 'cant',
 'fuck',
 'yes sir',
 'thanks',
 'nothing',
 'course',
 'unknown',
 'whats wrong',
 'hey',
 'uhhuh',
 'good',
 'thats right',
 'look',
 'im',
 'one',
 'love',
 'excuse',
 'thats',
 'didnt',
 'time',
 'shit',
 'oh god',
 'sorry',
 'dont think',
 'work',
 'way',
 'whats matter',
 'stop',
 'told',
 'guess',
 'dont understand',
 'long',
 'oh yeah',
 'help',
 'fine',
 'much',
 'would',
 'please',
 'jesus',
 'believe',
 'kill',
 'alright',
 'dad',
 'dont believe',
 'wait',
 'whats go',
 'name',
 'maybe',
 'whats name',
 'call',
 'great',
 'leave',
 'youre',
 'shut',
 'take',
 'ask',
 'understand',
 'could',
 'ok',
 'let go',
 'dead',
 'never',
 'hear',
 'bad',
 'give',
 'money',
 'kid',
 'whatd say

In [ ]:
messages = messages.drop_duplicates(subset=['msg_pre_processed'])

In [ ]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'yes']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,
817,L817,u9,m0,Yes,I don't like to do what people expect. Then th...,yes


Removing empty messages

In [ ]:

messages[messages['msg_pre_processed'] == '']

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,
50,L50,u3,m0,No.,You might wanna think about it,


In [ ]:
messages = messages.drop(messages[messages['msg_pre_processed'] == ''].index)

Removing nan msg origined by '' messages

In [ ]:
messages[messages['msg_pre_processed'].isna()]

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed
msg_line_clean,,,,,,


In [ ]:
#filling the nan messages with a string- not necessary
#messages = messages.fillna('-')
messages = messages.dropna()

Removing apostrophes (need for embedding and page rank dictionary)

In [ ]:
messages['msg_pre_processed'] = [ word.replace("\'","") for word in messages['msg_pre_processed'] ]

In [ ]:
# Filling '-' or '' messages with a generic one (not used)
# Tagging the msg with classes: 1 is question, 0 is answer

In [ ]:
def define_target(corpus):

    if '?' in corpus:
        return 1
    else:
        return 0

In [ ]:
data = messages['msg']

In [ ]:
messages['target'] = [define_target(m) for m in data]

In [ ]:

messages['target'] = messages['target'].astype(int)

Save data

In [ ]:
messages

,msg_line,user_id,movie_id,msg,msg_2,msg_pre_processed,target
msg_line_clean,,,,,,,
49,L49,u0,m0,Did you change your hair?,No.,change hair,1
59,L59,u9,m0,I missed you.,It says here you exposed yourself to a group o...,miss,0
60,L60,u8,m0,It says here you exposed yourself to a group o...,It was a bratwurst. I was eating lunch.,say expose group freshman girl,0
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?,bratwurst eat lunch,0
63,L63,u7,m0,You the new guy?,So they tell me...,new guy,1
...,...,...,...,...,...,...,...
666504,L666504,u9032,m616,Eleven o'clock My LorJ 42,Our friend Colonel Dumford will be should be a...,eleven oclock lorj 42,0
666520,L666520,u9034,m616,Well I assure you Sir I have no desire to crea...,And I assure you you do not In fact I'd be obl...,well assure sir desire create difficulty 45,0
666521,L666521,u9030,m616,And I assure you you do not In fact I'd be obl...,So far only their scouts. But we have had repo...,assure fact id oblige best advice scout see,1


In [ ]:
messages.to_csv('/content/drive/MyDrive/movie-corpus/movie_lines_pre_processed_similarity.tsv', index=False, sep='\t', header=False)


In [ ]:
# saving a small dataset pre processed
messages[0:30000].to_csv('/content/drive/MyDrive/movie-corpus/movie_lines_pre_processed_for_test.tsv', index=False, sep='\t', header=False)